In [1]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from scipy import ndimage
from PIL import Image
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn import svm
from sklearn import preprocessing as prp
from helpers_img_my import *
from Post_processing import *
#from helpers_img import *
from sklearn import model_selection as modsel
from preprocessing_my import *

In [2]:
def post_processing(label,threshold,size_min,verbarg,horbarg,size_image):
    label = complete_lines(label,threshold)
    label = remove_isolated_connected_component(label,size_min)
    label = clean_garbage_vert(label,verbarg,size_image)
    label = clean_garbage_hor(label,horbarg,size_image)
    label = remove_isolated_connected_component(label,size_min)
    #label = complete_semilines(label,0.85, size_image)
    return label

In [34]:
# Settings
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
patch_size = 16 # each patch is 16*16 pixels
deg=2

root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"
files = os.listdir(image_dir)

n=len(files)
#n=4
k_fold=2 #number of folds in the k-fold cross-validation

ims = [load_image(image_dir + files[i]) for i in range(n)]
gt_ims = [load_image(gt_dir + files[i]) for i in range(n)]

seed = 1
np.random.seed(seed)

kf=modsel.KFold(n_splits=k_fold,shuffle=True)
kf.get_n_splits(ims)
splits=kf.split(ims)
    
ims=[add_features(ims[i]) for i in range(len(ims))]
ims,gt_ims = rotation(ims,gt_ims)
ims,gt_ims = flip(ims,gt_ims)
print(len(ims))

lambdas=np.logspace(-8,-5,3) # 1/lambda
thresh=np.linspace(0.35,0.55,3)
shape=(lambdas.shape[0],thresh.shape[0])
mean_f1=np.zeros(shape)

#for (train_id,test_id) in splits:
trains=[]
tests=[]
for (train_id,test_id) in splits:
    trains.append(train_id)
    tests.append(test_id)
    
print(trains)
print(tests)

ims = [img_crop(ims[i], patch_size, patch_size) for i in range(len(ims))]
gt_ims = [img_crop(gt_ims[i], patch_size, patch_size) for i in range(len(gt_ims))]

ims = np.asarray([ims[i][j] 
                    for i in range(len(ims)) 
                    for j in range(len(ims[i]))])

gt_ims = np.asarray([gt_ims[i][j] 
                    for i in range(len(gt_ims)) 
                    for j in range(len(gt_ims[i]))])

ims=[extract_features(ims[i]) for i in range(len(ims))]
ims = np.asarray([poly_features(ims[i],deg) for i in range(len(ims))])
gt_ims = np.asarray([value_to_class(np.mean(gt_ims[i])) for i in range(len(gt_ims))])

for id_lam,lam in enumerate(lambdas):
    print('lambda: '+str(id_lam))
    for id_t,t in enumerate(thresh):
        print('thresh: '+str(id_t))
        nb_f1_te = np.zeros(k_fold)
        ind=0
        #for (train_id,test_id) in splits:
        for train,test in zip(trains,tests):
            print('here')
            im_tr = np.zeros((len(train)*625*8,325))
            gt_tr = np.zeros((len(train)*625*8))
            inter = 0
            for idx in train:
                temps = [ims[625*idx + (625*n)*k: 625*(idx+1) + (625*n)*k] for k in np.arange(0,8)]
                gt_temps = [gt_ims[625*idx + (625*n)*k :625*(idx+1) + (625*n)*k] for k in np.arange(0,8)]
                for (temp,gt_temp) in zip(temps,gt_temps):
                    im_tr[625*inter:625*(inter+1),:]= temp
                    gt_tr[625*inter:625*(inter+1)]=gt_temp
                    inter = inter + 1
            
            im_te = np.zeros((len(train)*625,325))
            gt_te = np.zeros((len(train)*625))
            inter = 0
            for idx in test:
                im_te[625*inter:625*(inter+1),:] = ims[625*idx : 625*(idx+1)]
                gt_te[625*inter:625*(inter+1)] = gt_ims[625*idx : 625*(idx+1)]
                inter = inter + 1
           
            ridgereg = linear_model.Ridge(alpha=lam, normalize=True)
            ridgereg.fit(im_tr, gt_tr)

            Z_te = ridgereg.predict(im_te)
            Z_te=1*(Z_te>t)
            Z_pp=[]
            for i in range(len(gt_te)):
                Z_pp = Z_pp + post_processing(Z_te[i*625:(i+1)*625],18,9,3,3,25)

            nb_f1_te[ind]=compute_F1(gt_te, Z_pp)
            print(nb_f1_te[ind])
            ind=ind+1
        
        mean_f1[id_lam,id_t]=nb_f1_te.mean()
    
best=np.max(mean_f1)
ids=np.where(mean_f1==best)
best_lambda=lambdas[ids[0][0]]
best_thresh=thresh[ids[1][0]]

32
[array([0, 2]), array([1, 3])]
[array([1, 3]), array([0, 2])]
lambda: 0
thresh: 0
here
0.8635682158920539
here
0.8705882352941177
thresh: 1
here
0.6836363636363637
here
0.726950354609929
thresh: 2
here
0.5256410256410257
here
0.6551724137931034
lambda: 1
thresh: 0
here
0.8700906344410876
here
0.8774583963691377
thresh: 1
here
0.6775956284153005
here
0.7129798903107861
thresh: 2
here
0.5161290322580645
here
0.6589595375722542
lambda: 2
thresh: 0
here
0.8648648648648649
here
0.878698224852071
thresh: 1
here
0.6715596330275229
here
0.7145421903052065
thresh: 2
here
0.49327354260089684
here
0.6589595375722542


In [27]:
[ims[625*idx : 625*(idx+1) + (625*n)*k] for k in np.arange(0,8)][0].shape

(625, 325)

In [17]:
im_tr

array([[array([[1.00000000e+00, 2.01731011e-01, 1.99249394e-01, ...,
        5.59237256e-09, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 1.94270838e-01, 2.17922799e-01, ...,
        6.98176668e-09, 8.71606791e-07, 1.08811771e-04],
       [1.00000000e+00, 3.18887874e-01, 3.35677090e-01, ...,
        9.51435157e-08, 2.65303282e-05, 7.39785898e-03],
       ...,
       [1.00000000e+00, 3.17432607e-01, 3.09451602e-01, ...,
        2.33870514e-08, 1.30762938e-05, 7.31128760e-03],
       [1.00000000e+00, 1.97947309e-01, 2.38204663e-01, ...,
        1.56268795e-08, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 2.93321086e-01, 2.96185669e-01, ...,
        3.23195296e-08, 1.51082533e-05, 7.06258171e-03]]),
        array([[1.00000000e+00, 2.01731011e-01, 1.99249394e-01, ...,
        5.59237256e-09, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 1.94270838e-01, 2.17922799e-01, ...,
        6.98176668e-09, 8.71606791e-07, 1.08811771e-04],
       [1.00000000e+00, 3.

In [ ]:
mean_f1

In [ ]:
thres=np.linspace(0.3,0.8,11)

In [ ]:
zers=np.zeros((thres.shape[0],lambdas.shape[0]))
np.argmax(zers)

In [ ]:
thres.shape[0]

In [ ]:
matrix=np.asarray([[1,2,3],[4,5,6]])
matrix

In [ ]:
ids=np.where(matrix==3)
ids[1][0]

In [ ]:
root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"
files = os.listdir(image_dir)

#n=len(files)
n=4
k_fold=2 #number of folds in the k-fold cross-validation

ims = [load_image(image_dir + files[i]) for i in range(n)]

In [ ]:
[ims[idx:idx+2] for idx in range(3)]

In [ ]:
[ims[625*idx:625*(idx+1)+(625*n)*k] for idx in train for k in np.arange(0,8)]

In [ ]:
ims[625*1:625*(1+1)+(625*3)*5]